In [41]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use([
    'grid',
    'notebook'
])

In [42]:
h3_interval = [10, 50]
h4_interval = [20, 70]

u_interval = [0, 100]

k1, k2, k3 = 165.48, 16.46, 29.4
b2, b3 = -156.93, -83.93

A_vertices = []

Ts = 1

r = .31
mu = .4
sigma = .55
a4 = .3019

B = np.array([
    [0],
    [k1/a4]
])

In [43]:
h3 = np.linspace(*h3_interval, 1000)
h4 = np.linspace(*h4_interval, 1000)

R43 = k2*(h4 - h3) + b2

q_43 = (h4 - h3) / R43
q_out = k3 * np.sqrt(h3) + b3

a3_1 = np.cos(2.5*np.pi * (h4 - mu)) / (sigma * np.sqrt(2 * np.pi))
a3_2 = np.exp(-((h4 - mu)**2) / (2 * sigma**2))

a3 = ((3*r)/5) * (2.7*r - (a3_1 * a3_2))

Z1 = Ts/R43
Z2 = (Ts*q_out) / h3
Z3 = 1/a3

z1_min, z1_max = np.min(Z1), np.max(Z1)
z2_min, z2_max = np.min(Z2), np.max(Z2)
z3_min, z3_max = np.min(Z3), np.max(Z3)

z1_interval = [z1_min, z1_max]
z2_interval = [z2_min, z2_max]
z3_interval = [z3_min, z3_max]

for z1 in z1_interval:
    for z2 in z2_interval:
        for z3 in z3_interval:
            A = np.array([
                [-z1*z3 + 1, z3*(z1 - z2)],
                [z1/a4, -(z1/a4) + 1]
            ])

            A_vertices.append(A)

In [44]:
from scipy.io import savemat

for i in range(len(A_vertices)):
    A = A_vertices[i]
    savemat(f'./vertices/A{i + 1}.mat', {'A': A})

savemat(f'./vertices/B.mat', {'B': B})